# LangChain Cohere Agent using Tools

## Install Libraries

In [1]:
# ! pip install streamlit
# ! pip install qdrant-client
# ! pip install langchain
# ! pip install langchain-community
# ! pip install langchain-qdrant
# ! pip install langchain-cohere
# ! pip install tiktoken
# ! pip install cohere
# ! pip install bs4
# ! pip install markdown
# ! pip install wikipedia
# ! pip install pyowm
# ! pip install yfinance

## Imports for Tool Use and Keys
### API Keys
* https://cohere.com/
* https://www.alphavantage.co/
* https://qdrant.tech/
* https://openweathermap.org/
* https://tavily.com/
* https://polygon.io/


In [1]:
from IPython.display import HTML
import markdown
import warnings
import streamlit as st
import os
from langchain_cohere.llms import Cohere
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere import ChatCohere, create_cohere_react_agent
from langchain_community.agent_toolkits.polygon.toolkit import PolygonToolkit
from langchain_community.utilities.polygon import PolygonAPIWrapper
from langchain.agents import AgentExecutor
from cohere_tools import get_internet_search_tool, get_wikipedia_tool, get_openweathermap_tool
from yfinance_tools import get_business_summary_tool, get_ticker_history_tool, get_ticker_info_tool
from alpha_vantage_tools import (
    get_search_symbols_tool,
    get_exchange_rate_tool,
    # get_time_series_daily_tool,
    # get_time_series_weekly_tool,
    # get_market_news_sentimen_tool,
    # get_top_gainers_losers_tool,
)

warnings.filterwarnings("ignore")

ALPHAVANTAGE_API_KEY = st.secrets["ALPHAVANTAGE_API_KEY"]
os.environ["ALPHAVANTAGE_API_KEY"] = st.secrets["ALPHAVANTAGE_API_KEY"]

COHERE_API_KEY = st.secrets["COHERE_API_KEY"]
os.environ["COHERE_API_KEY"] = st.secrets["COHERE_API_KEY"]

QDRANT_HOST = st.secrets["QDRANT_HOST"]
QDRANT_API_KEY = st.secrets["QDRANT_API_KEY"]

OPEN_WEATHER_API_KEY = st.secrets["OPEN_WEATHER_API_KEY"]
os.environ["OPENWEATHERMAP_API_KEY"] = OPEN_WEATHER_API_KEY

POLYGON_API_KEY = st.secrets["POLYGON_API_KEY"]
os.environ["POLYGON_API_KEY"] = st.secrets["POLYGON_API_KEY"]

TAVILY_API_KEY = st.secrets["TAVILY_API_KEY"]
os.environ["TAVILY_API_KEY"] = st.secrets["TAVILY_API_KEY"]

## Invoke ChatCohere

In [12]:
chat = ChatCohere()
messages = [HumanMessage(content="knock knock")]
print(chat.invoke(messages))

content="Who's there?" additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '481912a6-f20a-4afa-8510-7ff8ada7602c', 'token_count': {'input_tokens': 68, 'output_tokens': 4}} response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '481912a6-f20a-4afa-8510-7ff8ada7602c', 'token_count': {'input_tokens': 68, 'output_tokens': 4}} id='run-6264a1de-6ea7-43a2-adcf-e67617c0b032-0'


## Invoke ChatCohere using ChatPromptTemplate

In [13]:
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | chat
chain.invoke({"topic": "rabbits"})

AIMessage(content='Why did the rabbit go to the dance?\n\nHe wanted to hop around!', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '1943673b-0426-4f70-a693-be8e4f305c30', 'token_count': {'input_tokens': 72, 'output_tokens': 17}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '1943673b-0426-4f70-a693-be8e4f305c30', 'token_count': {'input_tokens': 72, 'output_tokens': 17}}, id='run-65cafa38-72d6-4658-b397-7107fb027bf1-0')

## Invoke Cohere

In [5]:
llm = Cohere()
display(HTML(markdown.markdown(llm.invoke("Summarize the theory of Kahnemann and Tversky"))))

## Invoke Cohere with Tools

In [2]:
def invoke_cohere_with_tools(input, tools, prompt_template="{input}",no_markdown=False,verbose=False):
    llm = ChatCohere()
    prompt = ChatPromptTemplate.from_template(prompt_template)
    agent = create_cohere_react_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=verbose)
    response = agent_executor.invoke(input)
    if no_markdown:
        print(response['output'])
    else:
        display(HTML(markdown.markdown(response['output'])))
    
    return response



def display_response_output(response,no_markdown=True):
    if no_markdown:
        print(response)
    else:
        display(HTML(markdown.markdown(response['output'])))
    
def display_response_citations(response,no_markdown=True):
    if no_markdown:
        print(response['citations'])
    else:
        display(HTML(markdown.markdown(response['citations'])))



## Tavily Internet Search Tool

In [16]:
prompt_template = "In which year had {stock_name} its IPO? Tell me the year and the date."
input =  {"stock_name": "Oracle"}
tools = [get_internet_search_tool()]
invoke_cohere_with_tools(input, tools, prompt_template)

## Wikipedia Tool

In [8]:
input =  {"input": "Explain the theory of Kahnemann and Tversky"}
tools = [get_wikipedia_tool()]
invoke_cohere_with_tools(input, tools)

## Run tool without agent

In [9]:
wikipedia_tool = get_wikipedia_tool()
wikipedia_tool.run("David Beckham")

"Page: David Beckham\nSummary: David Robert Joseph Beckham  ( BEK-əm; born 2 May 1975) is an English former professional footballer, the president and co-owner of Inter Miami CF and co-owner of Salford City. Primarily a right winger and known for his range of passing, crossing ability and free-kick taking, Beckham has been hailed as one of the greatest and most recognisable midfielders of his generation, as well as one of the best free-kick takers of all time. He won 19 major trophies in his career, and is the first English player to win league titles in four different countries: England, Spain, the United States and France.\nBeckham's professional club career began with Manchester United, where he made his first-team debut in 1992 at age 17. With United, he won the Premier League title six times, the FA Cup twice, the FA Charity Shield twice, the Intercontinental Cup and the UEFA Champions League in 1999. He then played four seasons with Real Madrid, winning the La Liga championship i

## Openweathermap Tool

In [17]:
input = {"input": "How is the weather in Salvador da Bahia?"}
tools = [get_openweathermap_tool()]
invoke_cohere_with_tools(input, tools, verbose=True)



> Entering new AgentExecutor chain...

I will search for the weather in Salvador da Bahia and then write an answer.
{'tool_name': 'open_weather_map', 'parameters': {'location': 'Salvador da Bahia'}}
In Salvador da Bahia, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 3.36 m/s, direction: 105°
Humidity: 77%
Temperature: 
  - Current: 23.98°C
  - High: 23.98°C
  - Low: 23.98°C
  - Feels like: 24.44°C
Rain: {}
Heat index: None
Cloud cover: 100%Relevant Documents: 0
Cited Documents: 0
Answer: In Salvador da Bahia, it is currently 23.98°C and overcast with 77% humidity. The wind speed is 3.36 m/s and it is blowing in a south-easterly direction.
Grounded answer: In Salvador da Bahia, it is currently <co: 0>23.98°C</co: 0> and <co: 0>overcast</co: 0> with <co: 0>77% humidity</co: 0>. The <co: 0>wind speed is 3.36 m/s</co: 0> and it is blowing in a <co: 0>south-easterly direction</co: 0>.

> Finished chain.


## Multiple Tool Use

1. wiki-tool
2. open_weather_map
3. internet_search

In [19]:
multiple_tools = [get_openweathermap_tool(), get_internet_search_tool(), get_wikipedia_tool()]
prompt_template = "Use the right tool to answer the following question. If wikipedia_tool delivers no result use internet_search: {input}"

input = {"input": "Who was George Washington?"}
invoke_cohere_with_tools(input, multiple_tools, prompt_template, verbose=True)

input = {"input": "What's the weather like in Mortugaba, Bahia, Brazil?"}
invoke_cohere_with_tools(input, multiple_tools, prompt_template, verbose=True)

input = {"input": "Whats in the news today?"}
invoke_cohere_with_tools(input, multiple_tools, prompt_template, verbose=True)



> Entering new AgentExecutor chain...

I will use Wikipedia to answer this question.
{'tool_name': 'wikipedia-tool', 'parameters': {'query': 'George Washington'}}
Page: George Washington
Summary: George Washington (February 22, 1732 – December 14, 1799) was an American Founding Father, military officer, and politician who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".
Washington's first public office, from 1749 to 1750, was as surveyor of Culpeper County in the Colony of Virginia. He subsequently received military training and was assigned command of the Virginia Regiment during the French and In



> Entering new AgentExecutor chain...

I will use the Open Weather Map tool to find out what the weather is like in Mortugaba, Bahia, Brazil.
{'tool_name': 'open_weather_map', 'parameters': {'location': 'Mortugaba, Bahia, Brazil'}}
In Mortugaba, Bahia, Brazil, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 3.13 m/s, direction: 117°
Humidity: 94%
Temperature: 
  - Current: 16.93°C
  - High: 16.93°C
  - Low: 16.93°C
  - Feels like: 17.13°C
Rain: {}
Heat index: None
Cloud cover: 100%Relevant Documents: 0
Cited Documents: 0
Answer: It's currently 16.93°C in Mortugaba, Bahia, Brazil. The weather is overcast with 100% cloud cover and 94% humidity.
Grounded answer: It's currently <co: 0>16.93°C</co: 0> in Mortugaba, Bahia, Brazil. The weather is <co: 0>overcast</co: 0> with <co: 0>100% cloud cover</co: 0> and <co: 0>94% humidity</co: 0>.

> Finished chain.




> Entering new AgentExecutor chain...

I will search for 'news 5 June 2024' and write an answer based on the results.
{'tool_name': 'internet_search', 'parameters': {'query': 'news 5 June 2024'}}
[{'url': 'https://www.nytimes.com/issue/todaysheadlines/2024/06/05/todays-headlines', 'content': 'Headlines from The New York Times for Wednesday, June 5, 2024. Top News. In Shift, Biden Issues Order Allowing Temporary Border Closure to Migrants. By Zolan Kanno-Youngs and Hamed Aleaziz.'}, {'url': 'https://www.cnn.com/2024/06/04/us/5-things-to-know-for-june-4-hunter-biden-india-election-covid-19-heat-dome-middle-east/index.html', 'content': "CNN's 5 Things brings you the news you need to know every morning. Ad Feedback. US. Crime + Justice ... Published 6:40 AM EDT, Tue June 4, 2024 Link Copied! Follow: Biden family See ..."}, {'url': 'https://wng.org/podcasts/wednesday-morning-news-june-5-2024-1717511922', 'content': 'WORLD Radio - Wednesday morning news: June 5, 2024. News of the day, incl

## Yfinance Tools

In [3]:
yfinance_tools = [get_business_summary_tool(),get_ticker_history_tool(), get_ticker_info_tool()]

### Business Summary Tool

In [13]:
prompt_template_summary = "Give me a business summary for {ticker}." 
input = {"ticker": "MSFT"}
invoke_cohere_with_tools(input=input,tools=yfinance_tools,prompt_template=prompt_template_summary)

### Ticker History Tool

In [14]:
prompt_template_history = "How did {ticker} perform the last {period}? Show the timeseries in a dataframe and calculate and rate the performance."
ticker = 'AMZN'
period = '5d'
input = {"ticker": {ticker}, "period": {period}}
invoke_cohere_with_tools(input=input, tools=yfinance_tools,prompt_template=prompt_template_history)

### Ticker Info Tool with Investment Rating

In [15]:
prompt_template_ticker_info = """I need detailed information about {ticker_symbol}. Produce a key value list for the 25 most important {topics} using CRLF before each point.
                                 After that analyze the fundamental and technical data and rate the company. Is it a good buy? Is it oversold or overbought? Is it a stable investment
                                 with a good sharpe ratio?"""
yfinance_tools = [get_ticker_info_tool()]
input = {"ticker_symbol": "AMSC", "topics": "key figures, technical indicators and analyst recommendations"}

invoke_cohere_with_tools(input=input, tools=yfinance_tools, prompt_template=prompt_template_ticker_info)

## Alpha Vantage Tool

In [21]:
yfinance_tools = [
    # get_search_symbol_tool(),
    get_exchange_rate_tool(),
    # get_time_series_daily_tool(),
    # get_time_series_weekly_tool(),
    # get_market_news_sentimen_tool(),
    # get_top_gainers_losers_tool(),
]
# alpha_vantage = AlphaVantageAPIWrapper()
# alpha_vantage._get_exchange_rate("USD", "JPY")
# alpha_vantage._get_time_series_daily("MSFT")
# alpha_vantage._get_time_series_weekly("IBM")
# alpha_vantage.search_symbols("American Superconductor")
# alpha_vantage._get_market_news_sentiment("Microsoft")
# alpha_vantage._get_top_gainers_losers()
# response = agent_executor.invoke(
#     {"input": "What is the symbol for American Superconductor?"}
# )
input = {"input": "How much is one USD in EUR?"}
invoke_cohere_with_tools(input, yfinance_tools, no_markdown= True, verbose=True)
# response = agent_executor.invoke({"input": "Who are the gainers and losers at the stock exchange?"})
# response = agent_executor.invoke({"input": "How is Microsoft performing historically?"})
# response = agent_executor.invoke({"input": "How did Apple perform the last time?"})
# response = agent_executor.invoke({"input": "What is in the news about Nvidia?"})



> Entering new AgentExecutor chain...

I will use the currency conversion tool to find out how much one USD is in EUR.
{'tool_name': 'av_get_exchange_rate', 'parameters': {'from_currency': 'USD', 'to_currency': 'EUR'}}
{'Realtime Currency Exchange Rate': {'1. From_Currency Code': 'USD', '2. From_Currency Name': 'United States Dollar', '3. To_Currency Code': 'EUR', '4. To_Currency Name': 'Euro', '5. Exchange Rate': '0.91920000', '6. Last Refreshed': '2024-06-05 09:52:32', '7. Time Zone': 'UTC', '8. Bid Price': '0.91917000', '9. Ask Price': '0.91920000'}}


> Finished chain.
{'Realtime Currency Exchange Rate': {'1. From_Currency Code': 'USD', '2. From_Currency Name': 'United States Dollar', '3. To_Currency Code': 'EUR', '4. To_Currency Name': 'Euro', '5. Exchange Rate': '0.91920000', '6. Last Refreshed': '2024-06-05 09:52:32', '7. Time Zone': 'UTC', '8. Bid Price': '0.91917000', '9. Ask Price': '0.91920000'}}


## Polygon Toolkit

### Polygon Tools

#### Get Tools

In [4]:
polygon = PolygonAPIWrapper()
toolkit = PolygonToolkit.from_polygon_api_wrapper(polygon)
polygon_tools = toolkit.get_tools()

#### Display Tools

In [6]:
for tool in polygon_tools:
    schema = str(tool.args_schema.schema())
    tool_documentation = f"<h1>{tool.name}</h1>\n\n<h2>Description</h2>{tool.description}\n\n<h2>Schema</h2>{schema}"
    display(HTML(markdown.markdown(tool_documentation)))

### Financials

In [7]:
prompt_template_av_financials = """How are the financials for {ticker}? What is its market value? 
                                   Produce an overview in form of a bullet point list (use CRLF for new lines)
                                   and rate the financial situation of the company."""
ticker = "AMSC"
input = {"ticker": ticker}
invoke_cohere_with_tools(tools=polygon_tools,input=input,prompt_template=prompt_template_av_financials)

### Ticker News Sentiment

In [ ]:
prompt_template_av_sentiment = "What is the news sentiment for {ticker}? Rate the sentiment based on the news and explain your rating."
ticker = "AMSC"
input = {"ticker": ticker}
invoke_cohere_with_tools(tools=polygon_tools,input=input,prompt_template=prompt_template_av_sentiment)

### Aggregates

#### Invoke LLM with Tool

In [ ]:
# Poloygon is called with the correct arguments but this produces an API error
# input = {"input": "How did MSFT perform the last 5 days?"}
# invoke_cohere_with_tools(tools=tools,input=input)

#### Run Tool

In [7]:
from langchain_community.tools.polygon.aggregates import PolygonAggregates
from langchain_community.tools.polygon.financials import PolygonFinancials
from langchain_community.tools.polygon.last_quote import PolygonLastQuote
from langchain_community.tools.polygon.ticker_news import PolygonTickerNews
from langchain_community.utilities.polygon import PolygonAPIWrapper
from langchain_community.tools.polygon.aggregates import PolygonAggregatesSchema

api_wrapper = PolygonAPIWrapper()
ticker = "AAPL"

# Define param
params = PolygonAggregatesSchema(
    ticker=ticker,
    timespan="day",
    timespan_multiplier=1,
    from_date="2024-03-01",
    to_date="2024-03-08",
)
# Get aggregates for ticker
aggregates_tool = PolygonAggregates(api_wrapper=api_wrapper)
aggregates = aggregates_tool.run(tool_input=params.dict())
print(aggregates)

[{"v": 73450582.0, "vw": 179.0322, "o": 179.55, "c": 179.66, "h": 180.53, "l": 177.38, "t": 1709269200000, "n": 911077}, {"v": 81505451.0, "vw": 174.8938, "o": 176.15, "c": 175.1, "h": 176.9, "l": 173.79, "t": 1709528400000, "n": 1167166}, {"v": 94702355.0, "vw": 170.3234, "o": 170.76, "c": 170.12, "h": 172.04, "l": 169.62, "t": 1709614800000, "n": 1108820}, {"v": 68568907.0, "vw": 169.5506, "o": 171.06, "c": 169.12, "h": 171.24, "l": 168.68, "t": 1709701200000, "n": 896297}, {"v": 71763761.0, "vw": 169.3619, "o": 169.15, "c": 169, "h": 170.73, "l": 168.49, "t": 1709787600000, "n": 825405}, {"v": 76267041.0, "vw": 171.5322, "o": 169, "c": 170.73, "h": 173.7, "l": 168.94, "t": 1709874000000, "n": 925213}]


### Last Quote

In [9]:
# Works only with premium subscription at Polygon
# input = {"input": "What is the end of day price for MSFT?"}
# invoke_cohere_with_tools(tools=polygon_tools,input=input)

## Financials, News and Technical Combined

In [8]:
combined_tools = []
filtered_polygon_tools = [tool for tool in polygon_tools if tool.name in ("polygon_financials", "polygon_ticker_news")]

combined_tools = filtered_polygon_tools

for tool in yfinance_tools:
    combined_tools.append(tool)
    
combined_tools.append(get_internet_search_tool())    
    
prompt_start = """0. I want you to act as an investment advisor.
                     Your advise is always reflected and based on sience and proven economic and investment theories.
                     Think step-by-step and use the right tools and check your results against this instruction.
                     Before analyzing make yourself a picture of 1. the current macroeconomic situation,
                     2. the actual performance of the big indices, 3. the performance of the different sectors, 4. the actual value of the fear and greed index and 5. the sentiment
                     at the stock market exchanges by doing a profound internet research.
                     Use therefore the tools of your choice.
                     Use always the right tools to answer each question.
                     After answering each question, make a final analysis as described in step 7.
                     Display the final analysis first and the detailed analysis from the steps 0 to 6 
                     before after the final analysis with the following headers 
                     overall situation,
                     company summary,
                     fundamental analysis,
                     technical analysis 
                     and sentiment analysis.                                         
               """    

prompt_template_summary = " 1. Get a summary for {ticker}. " 

prompt_template_ticker_info = """ 2. I need detailed information about {ticker}. Produce a key value list for the 25 for your analysis most important {topics} using CRLF before each point.
                                 After that analyze the fundamental and technical data and rate the company. Is it a good buy? Is it oversold or overbought? Is it a stable investment
                                 with a good sharpe ratio? Is it risky or safe to hold stocks of {ticker}? Display your detailed analysis at the end."""


prompt_template_av_financials = """ 3. How are the financials for {ticker}? What is its market value?
                                    Produce an overview in form of a bullet point list and use CRLF for new lines
                                    and rate the financial situation of the company and display it. """


prompt_template_av_sentiment = " 4. What is the news sentiment for {ticker}? Rate the sentiment based on the news and explain your rating and display it. "



prompt_template_history = " 5. How did {ticker} perform the last {period}? Get the timeseries in a dataframe and calculate and rate the performance and display it. "



prompt_template_search = """ 6. Search news and articles in the internet what you find about {ticker} and judge the sentiment. """

prompt_template_rating = """ 7. Based on all relevant information about fundamental, technical and stentiment data 
                            determined in the steps before for {ticker}
                            and the overall macroeconomic situation and sentiment at the stock market exchange
                            produce now a profound and detailed investment and trading advice based on a factor analysis and your own ratings in your report.
                            The rating should have an explicit header and explain why the stock is a buy or not.
                            Make an educated advise for a trader about a reasonable profit-target above the stock price, a reasonable stop-loss under the stock price
                            and a reasonable position size all in currency USD if the stock is a buy.                            
                         """

prompt_template_combined = prompt_start + prompt_template_summary +  prompt_template_ticker_info + prompt_template_av_financials + \
                           prompt_template_search +  prompt_template_av_sentiment +  prompt_template_history + \
                           prompt_template_rating

ticker = "TER"
period = "1y"
topics = "key figures, fundamental indicators, technical indicators and analyst recommendations"
input = {"ticker": ticker, "topics": topics, "period": period}
response = invoke_cohere_with_tools(input=input,tools=combined_tools,prompt_template=prompt_template_combined,verbose=False)

In [7]:
combined_tools = []
filtered_polygon_tools = [tool for tool in polygon_tools if tool.name in ("polygon_financials", "polygon_ticker_news")]

combined_tools = filtered_polygon_tools
for tool in yfinance_tools:
    combined_tools.append(tool)

prompt_template_av_financials = """ How are the financials for {ticker}? What is its market value?
                                    Produce an overview in form of a bullet point list (use CRLF for new lines)
                                    and rate the financial situation of the company. """


prompt_template_av_sentiment = " What is the news sentiment for {ticker}? Rate the sentiment based on the news and explain your rating. "

prompt_template_ticker_info = """ I need detailed information about {ticker}. Produce a key value list for the 25 most important {topics} using CRLF before each point.
                                 After that analyze the fundamental and technical data and rate the company. Is it a good buy? Is it oversold or overbought? Is it a stable investment
                                 with a good sharpe ratio? """

prompt_template_history = " How did {ticker} perform the last {period}? Show the timeseries in a dataframe and calculate and rate the performance. "

prompt_template_summary = " Give me a business summary for {ticker}. " 

prompt_template_rating = "Based on the by the tools retrieved information for {ticker} produce a profound investment rating in your report with a header and explain why the stock is a buy or not."

prompt_template_combined = prompt_template_summary + prompt_template_av_financials + prompt_template_av_sentiment + prompt_template_history + prompt_template_ticker_info + prompt_template_rating

ticker = "MSFT"
period = "1y"
topics = "key figures, technical indicators and analyst recommendations"
input = {"ticker": ticker, "topics": topics, "period": period}
invoke_cohere_with_tools(input=input,tools=combined_tools,prompt_template=prompt_template_combined,verbose=False)

{'ticker': 'MSFT',
 'topics': 'key figures, technical indicators and analyst recommendations',
 'period': '1y',
 'output': "## Business Summary:\nMicrosoft Corporation develops and supports software, services, devices and solutions worldwide. The company offers a range of products and services including office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot. It also provides LinkedIn and dynamics business solutions, such as Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power apps, and power automate. The company's Intelligent Cloud segment offers server products and cloud services, such as azure and other cloud services, SQL and windows server, visual studio, system center, and related client access licenses. The More Personal Computing segment offers Windows, including windows OEM licensing and other non-volume licensing of the Windows operating system, as well as devices such as surf

In [6]:
print(prompt_template_combined)

 Give me a business summary for {ticker}.  How are the financials for {ticker}? What is its market value?
                                    Produce an overview in form of a bullet point list (use CRLF for new lines)
                                    and rate the financial situation of the company.  What is the news sentiment for {ticker}? Rate the sentiment based on the news and explain your rating.  How did {ticker} perform the last {period}? Show the timeseries in a dataframe and calculate and rate the performance.  I need detailed information about {ticker}. Produce a key value list for the 25 most important {topics} using CRLF before each point.
                                 After that analyze the fundamental and technical data and rate the company. Is it a good buy? Is it oversold or overbought? Is it a stable investment
                                 with a good sharpe ratio? Based on the by the tools retrieved information for {ticker} produce a profound investment ratin

## Imports for Embeddings and Qdrant

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.vectorstores import Qdrant
from langchain_community.vectorstores import Qdrant
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversation.memory import ConversationSummaryMemory
import qdrant_client

## Build Qdrant Vector Store

In [ ]:
# web_links = ["https://www.fielmann-group.com/ueber-uns/?",
#              "https://de.wikipedia.org/wiki/Fielmann",
#              "https://de.wikipedia.org/wiki/G%C3%BCnther_Fielmann",
#              "https://de.wikipedia.org/wiki/Marc_Fielmann"] 

web_links = ["https://python.langchain.com/v0.2/docs/introduction",
             "https://python.langchain.com/v0.2/docs/concepts",
             "https://python.langchain.com/v0.2/docs/integrations/tools",
             "https://python.langchain.com/v0.2/docs/integrations/vectorstores",
             "https://python.langchain.com/v0.2/docs/integrations/document_loaders",
             "https://python.langchain.com/v0.2/docs/integrations/retrievers",
             "https://python.langchain.com/v0.2/docs/integrations/vectorstores/qdrant",
             "https://python.langchain.com/v0.2/docs/integrations/providers/cohere/#react-agent",
             "https://de.wikipedia.org/wiki/Herz#:~:text=Das%20Herz%20(lateinisch%20Cor%2C%20griechisch,die%20Versorgung%20aller%20Organe%20sichert."]
 

loader = WebBaseLoader(web_links)
document=loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(document)

embeddings = CohereEmbeddings(model = "embed-english-v2.0")

qdrant = Qdrant.from_documents(
    document,
    embeddings,
    url=QDRANT_HOST,
    api_key=QDRANT_API_KEY,
    prefer_grpc=True,
    collection_name="rag_documents",
    force_recreate=True,
)


## Load Qdrant Vector Store

In [ ]:
client = qdrant_client.QdrantClient(
    url=QDRANT_HOST,
    api_key=QDRANT_API_KEY,
)
embeddings = CohereEmbeddings(model="embed-english-v2.0")
vector_store = Qdrant(
    client = client,
    collection_name = "rag_documents",
    embeddings = embeddings
)
print("connection established !")

## Use Qdrant Vector Store as RAG Retriever for Cohere

In [ ]:
prompt_template = """
You are a bot to answer questions from a document.
    
You will be given a context of the conversation made so far followed by a question, 
give the answer to the question using the context. 
The answer should be short, straight and to the point. If you don't know the answer, reply that the answer is not available.

Context: {context}

Question: {question}
Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = { "prompt" : PROMPT }

llm = ChatCohere()

template = (
        """Combine the chat history and follow up question into 
        a standalone question. 
        If chat hsitory is empty, use the follow up question as it is.
        Chat History: {chat_history}
        Follow up question: {question}"""
    )

prompt = PromptTemplate.from_template(template)

chain = ConversationalRetrievalChain.from_llm(     
    llm = llm,
    chain_type = "stuff",
    memory = ConversationSummaryMemory(llm = llm, memory_key='chat_history', input_key='question', output_key= 'answer', return_messages=True),
    retriever = vector_store.as_retriever(search_type="mmr"),
    condense_question_prompt = prompt,
    return_source_documents=False,
    combine_docs_chain_kwargs=chain_type_kwargs,
)
customer_prompt = "What are tools in LangChain?"


llm_response = chain.invoke(
                {
                    "context": chain.memory.buffer,
                    "question": customer_prompt,
                },
                return_only_outputs=True,
            )

display(HTML(markdown.markdown(llm_response['answer'])))
